# CEPEA - Diárias do Mercado - Web Scraping

Esse _script_ realizará o _web scraping_ no site do CEPEA capturando todos as publicações de "Diárias de Mercado" relacionados a Acúcar e Etanol.

## 1. Requisitos

Vamos usar o [Beautiful Soup](https://pypi.org/project/beautifulsoup4/) (`bsf4`) para realizar a leitura do HTML e o [requests](https://pypi.org/project/requests/) para acessar websites a partir do HTTP.

## 2. Bibliotecas

* Beautiful Soup (`bsf4`)
* `requests`
* `pandas`
* `os`
* `time`

In [3]:
# Carregamento das bibliotecas
import os
import pandas as pd
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

## 3. Carregamento do Banco de Dados

In [5]:
# Construir o caminho completo para o arquivo CSV
base_dir = os.getcwd()  # Diretório atual
data_dir = os.path.join(base_dir, '01-data')  # Diretório '01-data' dentro do diretório atual

# Carregar o DataFrame de Feed a partir do arquivo CSV
df_merge = pd.read_csv(os.path.join(data_dir, '2024-08-11 cepea_feed_rss_merge.csv'), encoding='utf-8-sig')

## 4. Função

Esta função itera a cada linha do banco de dados. A coluna URL armazena o destino a ser visitado.

In [7]:
def extract_texts_and_update_df(df, output_filename):
    """
    Itera sobre as URLs no DataFrame, extrai o texto desejado de cada URL e adiciona
    o texto extraído como uma nova coluna ao DataFrame. Em seguida, salva o DataFrame atualizado em um arquivo CSV.

    Parâmetros:
    - df: DataFrame contendo as URLs para processar.
    - output_filename: Nome do arquivo CSV de saída (incluindo o caminho, se necessário).
    """
    # Lista para armazenar os textos extraídos
    texts = []

    # Iterar sobre cada linha do DataFrame
    for index, row in df.iterrows():
        url = row['URL']

        # Definir um cabeçalho User-Agent
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36"
        }
        
        try:
            # Fazer a requisição HTTP para a página
            response = requests.get(url, headers=headers)
            
            # Verificar se a requisição foi bem sucedida
            if response.status_code == 200:
                # Parsear o conteúdo HTML da página com o BeautifulSoup
                soup = BeautifulSoup(response.content, 'html.parser')
                
                # Localizar o div que contém o texto de interesse
                text_div = soup.find('div', class_='imagenet-ma-b imagenet-fl imagenet-col-12 imagenet-table-responsiva')
                
                if text_div:
                    # Extrair o texto do div
                    text = text_div.get_text(strip=True)
                    texts.append(text)
                else:
                    texts.append("Texto não encontrado")
            else:
                print(f"Erro ao acessar o URL {url}: {response.status_code}")
                texts.append("Erro ao acessar a página")
        
        except Exception as e:
            print(f"Erro ao processar o URL {url}: {e}")
            texts.append("Erro ao processar a página")

    # Faz uma cópia para não dar o Warning
    df_copy = df.copy()

    # Adicionar a nova coluna 'Texto' ao DataFrame
    df_copy['Texto'] = texts

    # Caminho completo para o arquivo de exportação
    file_path = os.path.join("01-data", time.strftime('%Y-%m-%d') + ' ' + output_filename)

    # Salvar o DataFrame atualizado em um novo arquivo CSV
    df_copy.to_csv(file_path, index=False, encoding='utf-8-sig')

    # Exibir o DataFrame
    return(df_copy)

## 5. Execução

In [9]:
# Execução da função para obter as publicações em uma banco de dados único
df_databse = extract_texts_and_update_df(df_merge, "cepea_feed_database.csv")